In [2]:
%matplotlib inline
import  matplotlib
import matplotlib.pyplot as plt
import pystan

In [3]:
import numpy as np
import scipy.io as sio

In [4]:
df = sio.loadmat('snelson1d.mat')
X = df['X']
Y = df['Y']
X_t = df['tX']
Y_t = df['tY']

In [5]:
M = 20

X_u = np.linspace(0,6, M)[:, None]
X_u_noisy = X_u + 0.1*np.random.rand(*X_u.shape) 

In [96]:
stan_code_rff = """
data {
    int<lower=1> N;
    int<lower=1> M;
    vector [N]x;
    vector[N] y;
#     real <lower=1e-6> sigvar;
#     real <lower=1e-6> sigma;
}

transformed data {
    real jitter = 1e-8;
    vector[N] f_mean = rep_vector(0, N);
    vector[M] omega;
    
    for(i in 1:M){
        omega[i] = normal_rng(0, 1);
    }
    
}

parameters{
    real <lower=1e-4> sigvar;
    real<lower=1e-6> sigma;
    
    vector[M] b;

}

transformed parameters {
    vector[N] f;
    
    
    matrix[N, N] K;
    
    matrix[N, M] features;
    matrix[N, M] cosfeatures;
    matrix[N, M] sinfeatures;
    matrix[N, 2*M] fullfeatures;
    
    {
        features = x * omega';

         for(i in 1:N){
             for(j in 1:M){
                 cosfeatures[i,j] = cos(features[i,j] + b[j]);
                 sinfeatures[i,j] = sin(features[i,j] + b[j]);
                 fullfeatures[i,j] = cosfeatures[i,j];
                 fullfeatures[i,M+j] = sinfeatures[i,j];
             }
         }

        K = fullfeatures * fullfeatures' * square(sigvar)/M;
        
        // add jitter
        for(i in 1:N){
            K[i,i] = K[i,i] + jitter;
        }
        
        L = cholesky_decompose(K);
    }
}


model {

    sigvar ~ normal(0.85, 0.25);
    sigma ~ normal(0, 0.35);
    b ~ uniform(0, 2*pi());
    
    f ~ multi_normal_cholesky(f_mean, L);
    y ~ normal(f, sigma);
    
}

"""

In [95]:
stan_model_rff = pystan.StanModel(model_code=stan_code_rff)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_88992040a2103588fe8e6ba9f3cc9e09 NOW.


In [100]:
gp_snelson_data = {'N':200, 'x':X.flatten(), 'y':Y.flatten(), 'M':50 }

In [98]:
gp_snelson_data1 = {'N':200, 'x':X.flatten(), 'y':Y.flatten(), 'M':50, 'sigvar':1.0, 'sigma':0.4 }

In [101]:
fit_gpr_rff = stan_model_rff.sampling(data=gp_snelson_data, iter=400, chains=2)

/u/26/dhakaa1/unix/anaconda2/envs/stanpy35/lib/python3.5/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


RuntimeError: Initialization failed.